# Gio Vanni Elbert Wisman (0706022210034)

In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from mlxtend.preprocessing import TransactionEncoder
%pip install mlxtend==0.23.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [21]:
# load the data set ans show the first five transaction
df = pd.read_csv(r"https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv")
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [23]:
unique_items = set(np.ravel(df))
for i in df:
  unique_items.update(df[i].unique())
print(unique_items)

{'Meat', 'Eggs', 'Bread', 'Wine', 'Pencil', 'Bagel', 'Milk', nan, 'Diaper', 'Cheese'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [16]:

itemset = set(unique_items)
# encoding the feature
encodedValue = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for i in uncommons:
        labels[i] = 0
    for j in commons:
        labels[j] = 1
    encodedValue.append(labels)

print(labels)

{'Pencil': 0, 'Milk': 0, 'Diaper': 0, 'Cheese': 0, 'Meat': 1, 'Eggs': 1, 'Bread': 1, 'Wine': 1, 'Bagel': 1, nan: 1}


In [17]:
  # create new dataframe from the encoded features
encodeddf = pd.DataFrame(encodedValue)
  # show the new dataframe
encodeddf.head()

,Milk,Bagel,NaN,Meat,Eggs,Bread,Wine,Pencil,Diaper,Cheese
0,0,0,0,1,1,1,1,1,1,1
1,1,0,0,1,0,1,1,1,1,1
2,1,0,1,1,1,0,1,0,0,1
3,1,0,1,1,1,0,1,0,0,1
4,0,0,1,1,0,0,1,1,0,0


In [25]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
encodeddf = encodeddf.drop(encodeddf.columns[1], axis=1)
encodeddf.head()

,Milk,Eggs,Bread,Wine,Pencil,Diaper,Cheese
0,0,1,1,1,1,1,1
1,1,0,1,1,1,1,1
2,1,1,0,1,0,0,1
3,1,1,0,1,0,0,1
4,0,0,0,1,1,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [26]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules

freqpurchase = apriori(encodeddf, min_support=0.2, use_colnames=True)
freqpurchase.head()

C:\Users\GEOVANI\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.501587,(Milk)
1,0.438095,(Eggs)
2,0.504762,(Bread)
3,0.438095,(Wine)
4,0.361905,(Pencil)


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [27]:
assRules = association_rules(freqpurchase, metric="confidence", min_threshold=0.6)
assRules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
1,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
2,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
3,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754,0.330409


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

1. Forerunner Bolster
This measures how regularly the predecessor thing (A) shows up within the dataset. It tells us the recurrence of exchanges that contain the forerunner.

2. Resulting Bolster
This measures how frequently the consequent item (C) shows up within the dataset. It tells us the recurrence of exchanges that contain the ensuing.

3. Back (of the Run the show)
This measures how frequently both the antecedent (A) and the ensuing (C) show up together within the same exchange. It appears the by and large recurrence of the combined things.

4. Certainty
Certainty measures how likely the ensuing (C) is to happen when the predecessor (A) occurs. It appears how regularly the run the show A → C holds genuine within the dataset.

5. Lift
Lift measures the quality of the relationship between A and C by comparing how regularly they show up together to how frequently they would show up autonomously. A lift esteem more prominent than 1 shows a positive relationship.

6. Use
Use measures how much more as often as possible A and C happen together than would be anticipated on the off chance that they were autonomous. A use esteem of shows no relationship between the two things.

7. Conviction
Conviction measures how much the event of the predecessor (A) impacts the event of the ensuing (C). The next conviction esteem demonstrates a more grounded reliance between A and C.

Rundown
Predecessor & Ensuing Back:
Tell us how frequently each item (A or C) shows up within the dataset.
Back (of the Run the show):
Measures how often A and C show up together.
Certainty:
Appears the probability of C happening when A happens.
Lift:
Compares the watched recurrence of A and C beside their anticipated frequency if free.
Use:
Measures how much more regularly A and C show up together than anticipated by chance.
Conviction:
Demonstrates how unequivocally A and C are subordinate on each other, with higher values appearing more grounded reliance.
These terms offer assistance evaluate the quality and unwavering quality of rules in affiliation run the show mining, which is valuable in showcase wicker container investigation and other ranges of information mining. 